<a href="https://colab.research.google.com/github/JayeshK10/Mass-vs-NoMass-Classification/blob/main/Mask_vs_No_Mask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dependencies

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.image as mpimg
import shutil
import random
import pickle


from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score, classification_report

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from numpy import linalg as LA
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

import keras
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.utils import np_utils

import os
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

import cv2
import matplotlib.image as mpimg
import glob 
import shutil

# Loading Data

In [ ]:
face_data_path = '/content/drive/MyDrive/Data_set/Mini_Proj/AFDB_face_dataset/*/*'
mask_data_path = '/content/drive/MyDrive/Data_set/Mini_Proj/AFDB_masked_face_dataset/*/*'

def image_loader(folder):
  img_path = []
  for img in glob.glob(folder):
    img_path.append(str(img))
  return img_path

no_mask_images = image_loader(face_data_path)
mask_images = image_loader(mask_data_path)

In [ ]:
a = len(mask_images) if (len(no_mask_images) > len(mask_images)) else len(no_mask_images) 
no_mask_img = no_mask_images[0:a]
mask_img = mask_images[0:a]
print('Number of no_mask images' , len(no_mask_img))
print('Number of mask images' , len(mask_img))

Number of no_mask images 2203
Number of mask images 2203


In [ ]:
def feature_extraction (dir_path , label_value):
  data = []
  label = []
  #img_list = []
  for filename in dir_path:
    img = mpimg.imread(filename)
    if img is not None:
      #img_list.append(img)
      gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
      resized = cv2.resize(gray_img,(28, 28), interpolation=cv2.INTER_CUBIC)
      # cv2_imshow(resized)
      data.append(resized.flatten())
      label.append(label_value)
  return data , label

In [ ]:
data_no_mask , label_no_mask = feature_extraction(no_mask_img , 0)
data_mask , label_mask = feature_extraction(mask_img , 1)